In [ ]:
##11月14日
path_rsimg = "E:\开源\cg_20.tif"
path_vec = "E:\开源\subset_4.gpkg"

In [2]:
import rasterio as rio##读栅格
import geopandas as gdp##读史量
rsimg_rio = rio.open(path_rsimg)
print(rsimg_rio.meta)
##uint无符号整型在光谱计算时需要转化为整型
vec_gdp = gdp.read_file(path_vec)
##矢量为WGS84(经纬度)的投影，但是栅格为utm(m)，需要转换矢量为utm
vec_gdp_utm = vec_gdp.to_crs(32648)
vec_gdp_utm


{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 1490, 'height': 1631, 'count': 6, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 47N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(20.0, 0.0, 874420.0,
       0.0, -20.0, 2769600.0)}


,geometry


In [3]:
import matplotlib.pyplot as plt

In [4]:
from rasterio import mask

In [5]:
##返回裁剪后的栅格数据与地理参数
##完成裁剪
rsimg_sub, rsimg_sub_trans = mask.mask(rsimg_rio, vec_gdp_utm.geometry, crop = True, nodata = 0)
##可视化裁剪后栅格


ValueError: min() arg is an empty sequence

In [ ]:
fig, ax = plt.subplots(1,2,figsize = (10, 5))
vec_gdp_utm.plot(ax=ax[0])
ax[1].imshow(rsimg_sub[0],vmin=0, vmax = 3000)


In [ ]:
path_subset = "E:\开源\cg_20_subset.tif"
##写出数据：读、写都是open，裁剪后宽高变了，地理转换参数也变了(地理转换参数包括空间分辨率、角点坐标，其中角点坐标变了)
out_meta = rsimg_rio.meta
out_meta.update({
    'driver':'GTiff',
    'height':rsimg_sub.shape[1],
    'width':rsimg_sub.shape[2],
    'transform': rsimg_sub_trans
})
with rio.open(path_subset , 'w', **out_meta) as dset:
    dset.write(rsimg_sub)

In [ ]:
##像刚刚一样再裁剪一幅相邻影像

In [ ]:
path_subset1 = "E:\开源\cg_20_subset.tif"
path_subset2 = "E:\开源\cg_20_subset2.tif"

In [ ]:
##读入：
subs1_rio = rio.open(path_subset1)
subs2_rio = rio.open(path_subset2)


In [ ]:
subs1_rio.meta
subs2_rio.meta
##确认投影一致

In [ ]:
from rasterio.merge import merge
##拼接后的影像与其转换参数：
mosaic,out_trans = merge([subs1_rio, subs2_rio])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (10, 4))
ax[0].imshow(subs1_rio.read(1), vmin = 0, vamx = 3000)
ax[1].imshow(subs2_rio.read(1), vmin = 0, vamx = 3000)
ax[2].imshow(mosaic[0], vmin = 0, vamx = 3000)